In this notebook, we demonstrate how to slice an stl-file into a gwl file including 3D stitching. The main difficulty so far with slicing STL files is that I have not found an easy way to slice it block by block. I need to create a stack for the full figure first, possibly with a lower resolution (for reducing the storage requirements). After this lowres stack is made, I can then make small sub-blocks using the low-res version to guide me in the assignement of where the material is located. 

In this notebook, we also demonstrate how to do the block ordering when the object spans multiple Z-ranges. If Concorde is not available this could be done with the nearest neighbour code as well, but I have not implemented it yet.

There is one big issue with the resulting GWL-file though: it does not print correctly! Describe puts the higher blocks at the correct height, but when writing with Nanoscribe, everything is printed in the same plane.. => Need to debug this!

In [1]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
import TipSlicer
import os
import scipy.ndimage as im
from importlib import reload
import tables

In [3]:
# load STL mesh and get X,Y, Z range
stlName='StanfordDragonBinary.stl'
rootName='StanfordDragon'

lowresSlicing=1
lowresHashing=1

mesh,Zmins,Zmaxs=TipSlicer.loadStl(stlName,centred=False,mulDimension=100)
X,Y,Z=TipSlicer.getMeshXYZ(mesh,lowresHashing,lowresSlicing)
print('X: {} -> {}  Y: {} -> {}  Z: {} -> {}'.format(X[0].min(),X[0].max(),Y[:,0].min(),Y[:,0].max(),Z.min(),Z.max()))
print(X.shape,Y.shape,Z.shape)



X: -710 -> 710  Y: 177 -> 817  Z: 0.0 -> 972.0
(641, 1421) (641, 1421) (973,)


In [5]:
Zmins.max(),Zmaxs.max(),Zmins.min()

(969.13324, 970.85333, 0.0)

In [4]:
# Do a first low resolution full scale stack with a correct infill
lowresH5=rootName+'_lowres.h5'
TipSlicer.stlToStack(mesh,lowresH5,X,Y,Z,Zmins,Zmaxs)

In [29]:
FOVrad=150  # more or less standard value but should be input
FOVz=300
overlapZ=2

amZ=int((Z.max()-Z.min())/FOVz)+1
Zrange=(Z.max()-Z.min())/amZ
print(amZ,Zrange)

# loop over different Z-slices
centreCoords=[]
for k in range(amZ):
    Zstart=k*Zrange
    Zstop=(k+1)*Zrange+overlapZ

    # now get low-resolution projection figures for this Z-range
    Ground,X,Y=TipSlicer.getProjection(lowresH5,Zstart,Zstop)

    # doOptimalStitching or hexagonal/square and find blockcentres
    optPoints=TipSlicer.getOptimalStitchCentres(Ground,FOVrad,FOVrad,X,Y)

    centreCoords.append(np.column_stack((optPoints,np.ones(len(optPoints))*Zstart)))


4 243.0


In [30]:
Zgrower=1000
newPPlist=[]
for cc in centreCoords:
    newPPlist.extend(cc*np.array([1,1,Zgrower]))
newPP=np.asarray(newPPlist)

TipSlicer.writeTSPfile('blocks.tsp',newPP)
!concorde -o 'tsp.sol' 'blocks.tsp'
Ord=TipSlicer.readTSPfile('tsp.sol')
ord3DCentres=(newPP[Ord]*np.array([1,1,1/Zgrower]))

if (ord3DCentres[0,2]>ord3DCentres[-1,2]):
    ord3DCentres=ord3DCentres[::-1]

concorde -o tsp.sol blocks.tsp
Host: lapsam  Current process id: 6908
Using random seed 1510664436
Problem Name: blocks
Problem Type: TSP
Nanoscribe Blocks in Ham Path
Number of Nodes: 46
Explicit Lengths (CC_MATRIXNORM)
Set initial upperbound to 738618 (from tour)
  LP Value  1: 10358.500000  (0.01 seconds)
  LP Value  2: 738601.000000  (0.01 seconds)
  LP Value  3: 738615.500000  (0.03 seconds)
New lower bound: 738615.500000
Final lower bound 738615.500000, upper bound 738618.000000
Exact lower bound: 738615.500000
DIFF: 0.000000
Time for Total: 0.08 seconds (0.08 total in 1 calls)
Final LP has 65 rows, 81 columns, 273 nonzeros
LOWER BOUND: 738615.500000   ACTIVE NODES: 1

Task 0: Branching on node 0
infeas_recover ...
phase 1 addbad_variables
Could not recover a feasible LP
Branched-LP is infeasible
phase 1 pricing
Problem is shown to be infeasible
BBnode 0 split into 1 (1000000000000000019884624838656.00X) 2 (738623.00X) (0.03 seconds)
Child 0 is infeasible
Child 1 is pruned

Task 

In [31]:
ord3DCentres.shape

(45, 3)

In [9]:
clf()
scatter(ord3DCentres[:,0],ord3DCentres[:,1],s=ord3DCentres[:,2])
plot(ord3DCentres[:,0],ord3DCentres[:,1],'r-')

In [34]:
clf()
#scatter(ord3DCentres[:,0],ord3DCentres[:,1],s=ord3DCentres[:,2])
plot(ord3DCentres[:,0],ord3DCentres[:,2],'r.-')

# now continue creating blocks

In [43]:
zstacks=np.unique(ord3DCentres[:,2])
zstacks

array([   0.,  243.,  486.,  729.])

In [57]:
stackI=np.argwhere(np.abs(ord3DCentres[:,2]-243)<1)[:,0]
optPoints=ord3DCentres[stackI,:2]

In [58]:
optPoints=ord3DCentres[stackI,:2]
optPoints.shape

(11, 2)

In [18]:
rootName='StanfordDragonNew'

In [20]:
# write the stacks for the different stitching blocks, will convert these to gwl later.

slicing=1

hashing=1
overlapXY_multiplier=3 # Final overlap will be 2 times this multiplied with hashing

if (not os.path.exists(rootName)):
    os.mkdir(rootName)
genBlockName=rootName+'/'+rootName+'Block-Layer {:05d} - nr {:05d}.h5'
genGwlName=rootName+'/'+rootName+'Block-Layer {:05d} - nr {:05d}.gwl'
genRegex=rootName+'/.+gwl'
lowresH5='StanfordDragon_lowres.h5'


# now creates the stacks
zstacks=np.unique(ord3DCentres[:,2])



for k in np.arange(len(zstacks)):
    Zstart=zstacks[k]
    Zstop=Zstart+Zrange
    zz=np.arange(Zstart,Zstop+slicing,slicing)
    sliceIndList= np.arange(len(zz))# adapt for variable slicing

    stackI=np.argwhere(np.abs(ord3DCentres[:,2]-Zstart)<1)[:,0]
    optPoints=ord3DCentres[stackI,:2]
    # generate voronoi polygons
    regions, vertices = TipSlicer.makeVoronoiPolygons(optPoints)

    # do loop over blocks and write out the stacks (needs stl and low-res stack) 
    #                                        => CAN I USE formulaToStack and give stl-name and low-res stack as parameters??
    fullDist=[]
    zCentres=[]
    for i in np.arange(len(optPoints)):
#    for i in np.arange(2):
        # now make coordinate systems
        print('Writing block {} in layer {}'.format(i,k))
        xc=optPoints[i,0]
        yc=optPoints[i,1]

        polygon = vertices[regions[i]]

        am=np.int(np.round(2*FOVrad/hashing))+1
        xx=np.linspace(xc-FOVrad,xc+FOVrad,am)
        yy=np.linspace(yc-FOVrad,yc+FOVrad,am)
        Xloc,Yloc=np.meshgrid(xx,yy)

        MM=TipSlicer.makeVoronoiMask(polygon,Xloc,Yloc)
        # if you need overlap, do binary_dilation on the mask
        MM=im.binary_dilation(MM,iterations=overlapXY_multiplier) # overlap = 2*iterations*hatching

        # with these coordinate systems, apply stlToStack
        blockName=genBlockName.format(k,i)

        TipSlicer.stlToStack(mesh,blockName,Xloc,Yloc,zz[sliceIndList],Zmins,Zmaxs,inFillMaster=lowresH5,writingMask=MM)

Writing block 0 in layer 0
Writing block 1 in layer 0
Writing block 2 in layer 0
Writing block 3 in layer 0
Writing block 4 in layer 0
Writing block 5 in layer 0
Writing block 6 in layer 0
Writing block 7 in layer 0
Writing block 8 in layer 0
Writing block 9 in layer 0
Writing block 10 in layer 0
Writing block 11 in layer 0
Writing block 12 in layer 0
Writing block 13 in layer 0
Writing block 0 in layer 1
Writing block 1 in layer 1
Writing block 2 in layer 1
Writing block 3 in layer 1
Writing block 4 in layer 1
Writing block 5 in layer 1
Writing block 6 in layer 1
Writing block 7 in layer 1
Writing block 8 in layer 1
Writing block 9 in layer 1
Writing block 10 in layer 1
Writing block 0 in layer 2
Writing block 1 in layer 2
Writing block 2 in layer 2
Writing block 3 in layer 2
Writing block 4 in layer 2
Writing block 5 in layer 2
Writing block 6 in layer 2
Writing block 7 in layer 2
Writing block 8 in layer 2
Writing block 9 in layer 2
Writing block 10 in layer 2
Writing block 11 in la

In [1]:
#now convert all hdf5-stacks to gwl-files.

h5Regex=rootName+'/.+\d\.h5' # should end in a digit, so as to avoid taking the _col.h5 files..
                             # alternatively, I could take them, and just reject upon inspection of the h5 file..
fullGwlName=rootName+'.gwl'

amShells=0
ScaffStep=2 # for solid infill => 2*hatching is spacing here
hatchStep=1 # for contour distance and top-bottom infill, not used if amShells==1 and doTopBottom=False

doColourH5=False


fp=os.path.split(h5Regex)
if (fp[0]==''):
    fnames,fnumbs=TipSlicer.findFiles(h5Regex)
else:
    fnames,fnumbs=TipSlicer.findFiles(fp[1],fp[0])

for ff in fnames:
    # should decide on the sliceIndList here. It could differ from the stack, but this is difficult to determine here
    # especially if it needs to be in Z => which Z's are we doing here? Unknown in advance.
    gwlName=ff[:-3]+'.gwl'
    print('Writing gwl: {}'.format(gwlName))
    dists=TipSlicer.stackToGwl(ff,gwlName,amShells,ScaffStep,hatchStep
                               ,atAngles=-1,writeHeader=False,writeColourH5=doColourH5,doTopBottom=True)

if (len(dists)>0):
    if (len(fullDist)==0):
        fullDist=dists
    else:
        fullDist=fullDist+dists

# Write out regex-gwl to follow this path.
TipSlicer.writeGwlForRegex(fullGwlName,genRegex)#,CodeSpeeds,CodeIntensities)

NameError: name 'rootName' is not defined